In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName('practice3').master('yarn').getOrCreate()

In [3]:
spark

In [4]:
orders = spark.read.json('/public/retail_db_json/orders')
orders.show()

+-----------------+--------------------+--------+---------------+
|order_customer_id|          order_date|order_id|   order_status|
+-----------------+--------------------+--------+---------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|
|             5657|2013-07-25 00:00:...|       9|PENDING_PAYMENT|
|             5648|2013-07-25 00:00:...|      10|PENDING_PAYMENT|
|              918|2013-07-25 00:00:...|      11| PAYMENT_REVIEW|
|             1837|2013-07-25 00:00:...|      12|         CLOSED|
|         

In [5]:
orders.dtypes

[('order_customer_id', 'bigint'),
 ('order_date', 'string'),
 ('order_id', 'bigint'),
 ('order_status', 'string')]

In [6]:
order_items = spark.read.csv('/public/retail_db/order_items',inferSchema=True). \
toDF('oi_id','oi_order_id','oi_product_id','oi_quantity','oi_subtotal','oi_produtprice')
order_items.show()


+-----+-----------+-------------+-----------+-----------+--------------+
|oi_id|oi_order_id|oi_product_id|oi_quantity|oi_subtotal|oi_produtprice|
+-----+-----------+-------------+-----------+-----------+--------------+
|    1|          1|          957|          1|     299.98|        299.98|
|    2|          2|         1073|          1|     199.99|        199.99|
|    3|          2|          502|          5|      250.0|          50.0|
|    4|          2|          403|          1|     129.99|        129.99|
|    5|          4|          897|          2|      49.98|         24.99|
|    6|          4|          365|          5|     299.95|         59.99|
|    7|          4|          502|          3|      150.0|          50.0|
|    8|          4|         1014|          4|     199.92|         49.98|
|    9|          5|          957|          1|     299.98|        299.98|
|   10|          5|          365|          5|     299.95|         59.99|
|   11|          5|         1014|          2|      

In [7]:
customers = spark.read.csv('/public/retail_db/customers').toDF('customer_id','customer_fname','customer_lname','customer_email', \
                                                              'customer_password','customer_street','customer_city','customer_state','customer_zipcode')
customers.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [8]:
departments = spark.read.csv('/public/retail_db/departments').toDF('dept_id','dept_name')
departments.show()

+-------+---------+
|dept_id|dept_name|
+-------+---------+
|      2|  Fitness|
|      3| Footwear|
|      4|  Apparel|
|      5|     Golf|
|      6| Outdoors|
|      7| Fan Shop|
+-------+---------+



In [9]:
products = spark.read.csv('/public/retail_db/products'). \
toDF('product_id','prod_category_id','product_name','product_desc','product_price','product_image')
products.show()

+----------+----------------+--------------------+------------+-------------+--------------------+
|product_id|prod_category_id|        product_name|product_desc|product_price|       product_image|
+----------+----------------+--------------------+------------+-------------+--------------------+
|         1|               2|Quest Q64 10 FT. ...|        null|        59.98|http://images.acm...|
|         2|               2|Under Armour Men'...|        null|       129.99|http://images.acm...|
|         3|               2|Under Armour Men'...|        null|        89.99|http://images.acm...|
|         4|               2|Under Armour Men'...|        null|        89.99|http://images.acm...|
|         5|               2|Riddell Youth Rev...|        null|       199.99|http://images.acm...|
|         6|               2|Jordan Men's VI R...|        null|       134.99|http://images.acm...|
|         7|               2|Schutt Youth Recr...|        null|        99.99|http://images.acm...|
|         

**File Formats and Compressions**

**1. Top N orders for the day based on order revenue**

In [10]:
#df methods top 5
joins = orders.join(order_items,orders.order_id==order_items.oi_order_id). \
groupBy(date_format('order_date','EEE')).agg(round(sum('oi_subtotal'),2).alias('Revenue'))
joins.show()

+----------------------------+----------+
|date_format(order_date, EEE)|   Revenue|
+----------------------------+----------+
|                         Sun| 4846988.5|
|                         Mon|4558745.38|
|                         Thu|4977875.18|
|                         Sat|4946465.98|
|                         Wed|4904572.31|
|                         Fri|5172864.23|
|                         Tue|4915108.35|
+----------------------------+----------+



In [11]:
joins = orders.join(order_items,orders.order_id==order_items.oi_order_id). \
groupBy('order_date').agg(round(sum('oi_subtotal'),2).alias('Revenue'))
joins.show()

+--------------------+---------+
|          order_date|  Revenue|
+--------------------+---------+
|2013-08-13 00:00:...| 39874.52|
|2013-10-12 00:00:...| 82006.54|
|2013-11-15 00:00:...| 68376.09|
|2014-03-19 00:00:...| 66732.76|
|2014-04-26 00:00:...|119101.07|
|2013-09-16 00:00:...| 56720.41|
|2013-09-20 00:00:...| 82662.51|
|2013-12-31 00:00:...|123799.52|
|2013-09-06 00:00:...|140273.12|
|2014-06-15 00:00:...| 61027.73|
|2013-12-24 00:00:...| 89744.21|
|2014-01-07 00:00:...| 84361.59|
|2014-06-07 00:00:...| 98239.16|
|2013-10-14 00:00:...| 61613.75|
|2013-11-11 00:00:...|114429.75|
|2014-01-27 00:00:...| 72634.57|
|2014-01-29 00:00:...| 87500.16|
|2014-02-14 00:00:...| 79936.26|
|2014-04-15 00:00:...|104220.93|
|2014-04-22 00:00:...| 75229.12|
+--------------------+---------+
only showing top 20 rows



In [12]:
rnk= joins.withColumn('rnk',dense_rank().over(Window.partitionBy('order_date').orderBy(desc('Revenue'))))
rnk.show()

+--------------------+---------+---+
|          order_date|  Revenue|rnk|
+--------------------+---------+---+
|2013-08-13 00:00:...| 39874.52|  1|
|2013-10-12 00:00:...| 82006.54|  1|
|2013-11-15 00:00:...| 68376.09|  1|
|2014-03-19 00:00:...| 66732.76|  1|
|2014-04-26 00:00:...|119101.07|  1|
|2013-09-16 00:00:...| 56720.41|  1|
|2013-09-20 00:00:...| 82662.51|  1|
|2013-12-31 00:00:...|123799.52|  1|
|2013-09-06 00:00:...|140273.12|  1|
|2014-06-15 00:00:...| 61027.73|  1|
|2013-12-24 00:00:...| 89744.21|  1|
|2014-01-07 00:00:...| 84361.59|  1|
|2014-06-07 00:00:...| 98239.16|  1|
|2013-10-14 00:00:...| 61613.75|  1|
|2013-11-11 00:00:...|114429.75|  1|
|2014-01-27 00:00:...| 72634.57|  1|
|2014-01-29 00:00:...| 87500.16|  1|
|2014-02-14 00:00:...| 79936.26|  1|
|2014-04-15 00:00:...|104220.93|  1|
|2014-04-22 00:00:...| 75229.12|  1|
+--------------------+---------+---+
only showing top 20 rows



In [13]:
spark.stop()